# Walk-through of Dice example in _Think Bayes_

In this notebook, I implement the main method that compute the entirety of the Dice example from the book _Think Bayes_ by Allen Downey. 

However, I want to understand how exactly he computes the posterior, so I deconstruct the example to make his calculations more transparent.

First I construct a helper function to import his modules and I import my own modules,

In [10]:
def import_file(full_path_to_module):
    try:
        import os
        module_dir, module_file = os.path.split(full_path_to_module)
        module_name, module_ext = os.path.splitext(module_file)
        save_cwd = os.getcwd()
        os.chdir(module_dir)
        module_obj = __import__(module_name)
        module_obj.__file__ = full_path_to_module
        globals()[module_name] = module_obj
        os.chdir(save_cwd)
    except:
        raise ImportError

import_file('/Users/nickgiangreco/gmail_dropbox/Dropbox/thinkbayes/ThinkBayes-master/code/thinkbayes.py')

import thinkbayes
import numpy as np
np.set_printoptions(precision=3)
import pandas as pd
import_file('/Users/nickgiangreco/gmail_dropbox/Dropbox/thinkbayes/ThinkBayes-master/code/dice.py')

In the book, the main output for this problem is

In [2]:
import_file('/Users/nickgiangreco/gmail_dropbox/Dropbox/thinkbayes/ThinkBayes-master/code/dice.py')
dice.main()

After one 6
4 0.0
6 0.392156862745
8 0.294117647059
12 0.196078431373
20 0.117647058824
After more rolls
4 0.0
6 0.0
8 0.915845271969
12 0.080403425797
20 0.00375130223399


However, how he computes the posterior is not transparent. I'll break this down step by step.

First, we have some hypotheses about our world, which is that there are 5 n-sided die that we could roll. There's no preference in which one we roll, so each is equally likely to be rolled. Below I print a dictionary showing the **prior** probability of rolling each die, since we have no **prior** knowledge of die preference.

In [42]:
hypos = [4, 6, 8, 12, 20]
#methods used by Allen Downey in book
v = dice.Dice(hypos)
print(v.d)

{8: 0.2, 12: 0.2, 4: 0.2, 6: 0.2, 20: 0.2}


In our world, we roll a dice and get a 6. That's our data.

In [11]:
data = 6

If we roll a 6, it wouldn't make sure that we rolled a 4-sided die and got a 6. Obviously, given the die, the probability of hypotheses changes. This is the **likelihood** of the data given the hypotheses. 

In [13]:
tmp = []
for hypo in u.Values():
    like = u.Likelihood(data, hypo)
    print("\n")
    print("If I rolled a "+repr(hypo)+"-sided die, how does my prior probability change if I roll a "+repr(data)+"?")
    print(u.d.get(hypo, 0) * like)
    tmp.append(u.d.get(hypo, 0) * like)



If I rolled a 8-sided die, how does my prior probability change if I roll a 6?
0.025


If I rolled a 12-sided die, how does my prior probability change if I roll a 6?
0.0166666666667


If I rolled a 4-sided die, how does my prior probability change if I roll a 6?
0.0


If I rolled a 6-sided die, how does my prior probability change if I roll a 6?
0.0333333333333


If I rolled a 20-sided die, how does my prior probability change if I roll a 6?
0.01


Let's see if this makes sense. Now the likelihood of the data, rolling a 6, is 0 if we rolled a 4-sided die. This makes sense-can't get a 6 without a 6 on the die! But for every die with increasing sides, the likelihood gets smaller. This makes sense because there's more options to roll than just a 6. 

Something to keep in mind: One of the 3 axioms of probability is that the sum of probabilities has to equal one. Right now they don't sum to 1 and so we can't compare the likelihoods. Now we use the **normalizing constant** to make these probabilities again.

In [43]:
print('Sum of likelihoods '+repr(sum(tmp))+'\n')
tmp2 = []
for t in tmp:
    tmp2.append(t / sum(tmp))
print(pd.Series(tmp2,index=hypos))

Sum of likelihoods 0.08500000000000002

4     0.294118
6     0.196078
8     0.000000
12    0.392157
20    0.117647
dtype: float64


The above is exactly what Allen Downey computes in the book! Yay! 

Let me put this in a nice display so the process makes sense.

In [46]:
#instantiate dataframe
df = pd.DataFrame()
#make cols

#hypotheses
df['Hypothesis'] = [x for x,y in u.Items()]
#prior probability for hypotheses
df['Prior'] = [y for x,y in u.Items()]
#data
df['Data'] = np.repeat(6,df.shape[0])
#probability of data
df['P(Data)'] = np.repeat(1.0/6.0,df.shape[0])
#likelihood-probability of data given an hypothesis
df['Likelihood'] = tmp
#normalizing likelihoods to get posterior
df['Posterior'] = df['Likelihood'] / sum(tmp)
#sorting by hypotheses
df.sort_values(by=['Hypothesis'],inplace=False)

,Hypothesis,Prior,Data,P(Data),Likelihood,Posterior
2,4,0.2,6,0.166667,0.000000,0.000000
3,6,0.2,6,0.166667,0.033333,0.392157
0,8,0.2,6,0.166667,0.025000,0.294118
1,12,0.2,6,0.166667,0.016667,0.196078
4,20,0.2,6,0.166667,0.010000,0.117647
